In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pickle
import joblib
import dagshub
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from evidently import Dataset, DataDefinition
from evidently import Report
from evidently import BinaryClassification
from evidently.presets import DataDriftPreset, ClassificationPreset
from datetime import datetime
from sklearn.preprocessing import LabelEncoder

c:\Users\ajana\Music\Projects\Rock vs Mine\.venv\Lib\site-packages\mlflow\utils\requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


In [2]:
import evidently
print(evidently.__version__)

0.7.14


In [3]:
#dagshub.init(repo_owner='King-David02', repo_name='Rock-vs-Mine', mlflow=True)

In [4]:
data = pd.read_csv('data/Sonar.csv')
data

,Freq_1,Freq_2,Freq_3,Freq_4,Freq_5,Freq_6,Freq_7,Freq_8,Freq_9,Freq_10,...,Freq_52,Freq_53,Freq_54,Freq_55,Freq_56,Freq_57,Freq_58,Freq_59,Freq_60,Label
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,M
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,M
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,M
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,M


In [5]:
data= data.rename(columns={'Label': 'Target'})
data

,Freq_1,Freq_2,Freq_3,Freq_4,Freq_5,Freq_6,Freq_7,Freq_8,Freq_9,Freq_10,...,Freq_52,Freq_53,Freq_54,Freq_55,Freq_56,Freq_57,Freq_58,Freq_59,Freq_60,Target
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,M
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,M
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,M
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,M


In [6]:
le = LabelEncoder()
data['target_encoded'] = le.fit_transform(data['Target'])
data

,Freq_1,Freq_2,Freq_3,Freq_4,Freq_5,Freq_6,Freq_7,Freq_8,Freq_9,Freq_10,...,Freq_53,Freq_54,Freq_55,Freq_56,Freq_57,Freq_58,Freq_59,Freq_60,Target,target_encoded
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R,1
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R,1
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R,1
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R,1
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,M,0
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,M,0
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,M,0
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,M,0


In [7]:
print(data[['Target', 'target_encoded']].head(10))

  Target  target_encoded
0      R               1
1      R               1
2      R               1
3      R               1
4      R               1
5      R               1
6      R               1
7      R               1
8      R               1
9      R               1


In [8]:
drop = ['Target', 'target_encoded']
X = data.drop(drop, axis=1)
y = data['target_encoded']

In [9]:
X

,Freq_1,Freq_2,Freq_3,Freq_4,Freq_5,Freq_6,Freq_7,Freq_8,Freq_9,Freq_10,...,Freq_51,Freq_52,Freq_53,Freq_54,Freq_55,Freq_56,Freq_57,Freq_58,Freq_59,Freq_60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0203,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0051,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0155,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0042,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048


In [10]:
y

0      1
1      1
2      1
3      1
4      1
      ..
203    0
204    0
205    0
206    0
207    0
Name: target_encoded, Length: 208, dtype: int32

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
X_test

,Freq_1,Freq_2,Freq_3,Freq_4,Freq_5,Freq_6,Freq_7,Freq_8,Freq_9,Freq_10,...,Freq_51,Freq_52,Freq_53,Freq_54,Freq_55,Freq_56,Freq_57,Freq_58,Freq_59,Freq_60
161,0.0305,0.0363,0.0214,0.0227,0.0456,0.0665,0.0939,0.0972,0.2535,0.3127,...,0.0271,0.0200,0.0070,0.0070,0.0086,0.0089,0.0074,0.0042,0.0055,0.0021
15,0.0298,0.0615,0.0650,0.0921,0.1615,0.2294,0.2176,0.2033,0.1459,0.0852,...,0.0154,0.0031,0.0153,0.0071,0.0212,0.0076,0.0152,0.0049,0.0200,0.0073
73,0.0139,0.0222,0.0089,0.0108,0.0215,0.0136,0.0659,0.0954,0.0786,0.1015,...,0.0024,0.0062,0.0072,0.0113,0.0012,0.0022,0.0025,0.0059,0.0039,0.0048
96,0.0181,0.0146,0.0026,0.0141,0.0421,0.0473,0.0361,0.0741,0.1398,0.1045,...,0.0076,0.0223,0.0255,0.0145,0.0233,0.0041,0.0018,0.0048,0.0089,0.0085
166,0.0411,0.0277,0.0604,0.0525,0.0489,0.0385,0.0611,0.1117,0.1237,0.2300,...,0.0181,0.0217,0.0038,0.0019,0.0065,0.0132,0.0108,0.0050,0.0085,0.0044
9,0.0164,0.0173,0.0347,0.0070,0.0187,0.0671,0.1056,0.0697,0.0962,0.0251,...,0.0118,0.0090,0.0223,0.0179,0.0084,0.0068,0.0032,0.0035,0.0056,0.0040
100,0.0629,0.1065,0.1526,0.1229,0.1437,0.1190,0.0884,0.0907,0.2107,0.3597,...,0.0257,0.0089,0.0262,0.0108,0.0138,0.0187,0.0230,0.0057,0.0113,0.0131
135,0.0094,0.0611,0.1136,0.1203,0.0403,0.1227,0.2495,0.4566,0.6587,0.5079,...,0.0480,0.0234,0.0175,0.0352,0.0158,0.0326,0.0201,0.0168,0.0245,0.0154
18,0.0270,0.0092,0.0145,0.0278,0.0412,0.0757,0.1026,0.1138,0.0794,0.1520,...,0.0045,0.0084,0.0010,0.0018,0.0068,0.0039,0.0120,0.0132,0.0070,0.0088
148,0.0712,0.0901,0.1276,0.1497,0.1284,0.1165,0.1285,0.1684,0.1830,0.2127,...,0.0154,0.0154,0.0156,0.0054,0.0030,0.0048,0.0087,0.0101,0.0095,0.0068


In [13]:
pipeline = make_pipeline(StandardScaler(),
                         RandomForestClassifier(n_estimators=100, random_state=42))

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
sonar_data_prediction = pipeline.predict(data.drop(drop, axis=1))
joblib.dump(pipeline, 'model/pipeline.pkl')

['model/pipeline.pkl']

In [14]:
cm = confusion_matrix(y, sonar_data_prediction)
print(cm)

[[107   4]
 [  2  95]]


In [15]:
y_pred

array([0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1])

In [16]:
other_data = pd.read_csv('data/rock_vs_mine_dataset.csv')
other_data['target_encoded'] = le.transform(other_data['Target'])
other_data['prediction'] = pipeline.predict(other_data.drop(drop, axis=1))
other_data.to_csv('data/predicted_data/current_data.csv', index=False)

In [17]:
data['prediction'] = sonar_data_prediction
data.to_csv('data/predicted_data/reference_data.csv', index=False)

In [18]:
definition = DataDefinition(classification=[
    BinaryClassification(
        target='target_encoded',
        prediction_labels='prediction',
        #labels={0: 'Mine', 1: 'Rock'},
        pos_label=0
    )
])

In [19]:
reff_data = Dataset.from_pandas(data=data, data_definition=definition)
curr_data = Dataset.from_pandas(data=other_data, data_definition=definition)

In [20]:
report = Report(metrics=[
    DataDriftPreset(),
    ClassificationPreset()
])

In [21]:
report = report.run(current_data=curr_data, reference_data=reff_data)

In [22]:
from evidently.ui.workspace import Workspace
ws = Workspace.create('workspace')

In [23]:
project = ws.create_project('RvM_Dashboard')
project

Project ID: 01991e37-1ec7-7965-af46-07ccff981fe8
Project Name: RvM_Dashboard
Project Description: None
        

In [24]:
ws.add_run(project.id, report)

Report ID: 01991e37-1f07-72b6-aec6-a4ebf350529e
Link: workspace\01991e37-1ec7-7965-af46-07ccff981fe8\snapshots\01991e37-1f07-72b6-aec6-a4ebf350529e.json

In [25]:
'''st = StandardScaler()
X_train = st.fit_transform(X_train)
X_test = st.transform(X_test)'''

'st = StandardScaler()\nX_train = st.fit_transform(X_train)\nX_test = st.transform(X_test)'

In [26]:
'''rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
joblib.dump(st, "scaler.pkl")
joblib.dump(rf, "rf_model.pkl")'''

'rf = RandomForestClassifier(n_estimators=100, random_state=42)\nrf.fit(X_train, y_train)\ny_pred = rf.predict(X_test)\njoblib.dump(st, "scaler.pkl")\njoblib.dump(rf, "rf_model.pkl")'

In [27]:
'''with mlflow.start_run(run_name='Artifacts Method'):
    mlflow.autolog()
    st = StandardScaler()
    X_train = st.fit_transform(X_train)
    X_test = st.transform(X_test)
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    report_str = classification_report(y_test, y_pred)
    with open("classification_report.txt", "w") as f:
        f.write(report_str)
    mlflow.log_artifact("classification_report.txt")
    joblib.dump(st, "scaler.pkl")
    mlflow.log_artifact("scaler.pkl")
    joblib.dump(rf, "rf_model.pkl")
    mlflow.log_artifact("rf_model.pkl")
    #mlflow.sklearn.log_model(rf, name="model")'''

'with mlflow.start_run(run_name=\'Artifacts Method\'):\n    mlflow.autolog()\n    st = StandardScaler()\n    X_train = st.fit_transform(X_train)\n    X_test = st.transform(X_test)\n    rf = RandomForestClassifier(n_estimators=100, random_state=42)\n    rf.fit(X_train, y_train)\n    y_pred = rf.predict(X_test)\n    report_str = classification_report(y_test, y_pred)\n    with open("classification_report.txt", "w") as f:\n        f.write(report_str)\n    mlflow.log_artifact("classification_report.txt")\n    joblib.dump(st, "scaler.pkl")\n    mlflow.log_artifact("scaler.pkl")\n    joblib.dump(rf, "rf_model.pkl")\n    mlflow.log_artifact("rf_model.pkl")\n    #mlflow.sklearn.log_model(rf, name="model")'

In [28]:
'''with mlflow.start_run(run_name='Random Forest'):
    mlflow.autolog()
    pipeline = make_pipeline(StandardScaler(),
                            RandomForestClassifier()
                            )
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    sample_data = X[:10]
    sample_prediction = pipeline.predict(sample_data)
    signature = infer_signature(sample_data, sample_prediction)
    report_str = classification_report(y_test, y_pred)
    with open("classification_report.txt", "w") as f:
        f.write(report_str)
    mlflow.log_artifact("classification_report.txt")
    mlflow.sklearn.log_model(pipeline, "model", signature=signature)
    #joblib.dump(pipeline, "pipeline.pkl")
    #mlflow.log_artifact("pipeline.pkl")'''

'with mlflow.start_run(run_name=\'Random Forest\'):\n    mlflow.autolog()\n    pipeline = make_pipeline(StandardScaler(),\n                            RandomForestClassifier()\n                            )\n    pipeline.fit(X_train, y_train)\n    y_pred = pipeline.predict(X_test)\n    sample_data = X[:10]\n    sample_prediction = pipeline.predict(sample_data)\n    signature = infer_signature(sample_data, sample_prediction)\n    report_str = classification_report(y_test, y_pred)\n    with open("classification_report.txt", "w") as f:\n        f.write(report_str)\n    mlflow.log_artifact("classification_report.txt")\n    mlflow.sklearn.log_model(pipeline, "model", signature=signature)\n    #joblib.dump(pipeline, "pipeline.pkl")\n    #mlflow.log_artifact("pipeline.pkl")'

In [29]:
'''with open('model.pkl', 'wb') as file:
    pickle.dump(pipeline, file)'''

"with open('model.pkl', 'wb') as file:\n    pickle.dump(pipeline, file)"

In [30]:
#print(mlflow.get_tracking_uri())

In [31]:
'''with mlflow.start_run(run_name='Logistic'):
    mlflow.autolog()
    pipeline2 = make_pipeline(
        StandardScaler(),
        LogisticRegression()
    )
    pipeline2.fit(X_train, y_train)
    prediction = pipeline2.predict(X_test)
    sample_data = X[:10]
    sample_prediction = pipeline2.predict(sample_data)
    signature2 = infer_signature(sample_data, sample_prediction)
    report_str_lr = classification_report(y_test, prediction)
    with open("lr_classification_report.txt", "w") as f:
        f.write(report_str_lr)
    mlflow.log_artifact("lr_classification_report.txt")
    #joblib.dump(pipeline2, "lr_model.pkl")
    #mlflow.log_artifact("lr_model.pkl")
    #mlflow.sklearn.log_model(pipeline2, "lrmodel", signature=signature2)'''

'with mlflow.start_run(run_name=\'Logistic\'):\n    mlflow.autolog()\n    pipeline2 = make_pipeline(\n        StandardScaler(),\n        LogisticRegression()\n    )\n    pipeline2.fit(X_train, y_train)\n    prediction = pipeline2.predict(X_test)\n    sample_data = X[:10]\n    sample_prediction = pipeline2.predict(sample_data)\n    signature2 = infer_signature(sample_data, sample_prediction)\n    report_str_lr = classification_report(y_test, prediction)\n    with open("lr_classification_report.txt", "w") as f:\n        f.write(report_str_lr)\n    mlflow.log_artifact("lr_classification_report.txt")\n    #joblib.dump(pipeline2, "lr_model.pkl")\n    #mlflow.log_artifact("lr_model.pkl")\n    #mlflow.sklearn.log_model(pipeline2, "lrmodel", signature=signature2)'

In [32]:
import sys
print(sys.executable)

c:\Users\ajana\Music\Projects\Rock vs Mine\.venv\Scripts\python.exe
